<a href="https://colab.research.google.com/github/shashvindu/Pyspark/blob/main/pysaprksetup_testwithretaildata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
#sc.stop()

In [19]:
#!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [20]:
#!wget -q https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz

In [21]:
#!tar xf spark-3.3.0-bin-hadoop3.tgz

In [22]:
#!pip install -q findspark
#!pip install pyspark

In [23]:
from pyspark import SparkContext

In [24]:
sc=SparkContext()

In [25]:
a=sc.parallelize([1,2,3])

In [26]:
a.take(2)

[1, 2]

In [27]:
sc.stop()

In [28]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

retails_data_csv = "/content/drive/MyDrive/data/retail-data/by-day/*.csv"

In [29]:
spark = SparkSession \
    .builder \
    .appName("Basic Dataframe Operations") \
    .config("spark.master", "local") \
    .getOrCreate()

In [30]:
df = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load(retails_data_csv)\
  .repartition(5)

In [31]:
df.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   580729|    22492|MINI PAINT SET VI...|       1|2011-12-05 17:24:00|     1.63|      null|United Kingdom|
|   575070|    21756|BATH BUILDING BLO...|       1|2011-11-08 12:58:00|     6.25|   17841.0|United Kingdom|
|   574573|    22945|CHRISTMAS METAL T...|       6|2011-11-04 16:35:00|     0.85|   14592.0|United Kingdom|
|   575911|    35970|ZINC FOLKART SLEI...|      24|2011-11-11 16:13:00|     1.69|   17314.0|United Kingdom|
|   576559|    22360|GLASS JAR ENGLISH...|       1|2011-11-15 13:19:00|     2.95|   14711.0|United Kingdom|
|   569900|    35970|ZINC FOLKART SLEI...|      12|2011-10-06 16:17:00|     1.69|   17651.0|United Kingdom|
|   539453|    21365|MIRRORE

In [32]:
df.rdd.getNumPartitions()

5

In [33]:
#sql table
df.createOrReplaceTempView("dfTable")

In [34]:
from pyspark.sql.functions import count, countDistinct, approx_count_distinct
df.select(count("StockCode")).show()

+----------------+
|count(StockCode)|
+----------------+
|          541909|
+----------------+



In [35]:
df.groupBy("StockCode").count().show()

+---------+-----+
|StockCode|count|
+---------+-----+
|    22596|  274|
|    21249|  119|
|    21894|  135|
|    22121|  141|
|    23318|  391|
|    20868|   46|
|    22728|  810|
|    23459|   21|
|    90022|   21|
|    22254|   61|
|    21259|  296|
|    21452|  200|
|   90210B|    7|
|    90143|   22|
|   90197B|   27|
|    21248|   68|
|    21889|  607|
|   85132b|    9|
|    21331|    8|
|   85115B|    1|
+---------+-----+
only showing top 20 rows



In [36]:
df.select(countDistinct("StockCode")).show()

+-------------------------+
|count(DISTINCT StockCode)|
+-------------------------+
|                     4070|
+-------------------------+



In [37]:
df.select(approx_count_distinct("StockCode", 0.1)).show()

+--------------------------------+
|approx_count_distinct(StockCode)|
+--------------------------------+
|                            3364|
+--------------------------------+



In [38]:
spark.sql("""SELECT count(StockCode),
                    approx_count_distinct(StockCode)
             FROM dfTable""").show()

+----------------+--------------------------------+
|count(StockCode)|approx_count_distinct(StockCode)|
+----------------+--------------------------------+
|          541909|                            3804|
+----------------+--------------------------------+



In [39]:
sqlc="SELECT count(StockCode), approx_count_distinct(StockCode)  FROM dfTable"
spark.sql(sqlc).show()

+----------------+--------------------------------+
|count(StockCode)|approx_count_distinct(StockCode)|
+----------------+--------------------------------+
|          541909|                            3804|
+----------------+--------------------------------+



In [40]:
#  Example 2 - first, last, min, max
from pyspark.sql.functions import first, last, min, max

In [41]:
df.select(first("StockCode"), last("StockCode")).show()

+----------------+---------------+
|first(StockCode)|last(StockCode)|
+----------------+---------------+
|           22148|          21201|
+----------------+---------------+



In [42]:
df.select(min("Quantity"), max("Quantity")).show()

+-------------+-------------+
|min(Quantity)|max(Quantity)|
+-------------+-------------+
|       -80995|        80995|
+-------------+-------------+



In [43]:
spark.sql("""SELECT first(StockCode) as first, 
                    last(StockCode) as last,
                    min(Quantity)  as minQty,
                    max(Quantity) as maxQty
             FROM dfTable""").show()

+-----+-----+------+------+
|first| last|minQty|maxQty|
+-----+-----+------+------+
|22578|22641|-80995| 80995|
+-----+-----+------+------+



In [44]:
#  Example 3 - sum, sumDistinct, avg
from pyspark.sql.functions import sum, sumDistinct, avg

In [45]:
df.select(sum("Quantity")).show() 

+-------------+
|sum(Quantity)|
+-------------+
|      5176450|
+-------------+



In [46]:
df.select(sumDistinct("Quantity")).show()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/functions.py:315: FutureWarning: Deprecated in 3.2, use sum_distinct instead.
  warnings.warn("Deprecated in 3.2, use sum_distinct instead.", FutureWarning)


+----------------------+
|sum(DISTINCT Quantity)|
+----------------------+
|                 29310|
+----------------------+



In [47]:
spark.sql("""SELECT sum(Quantity)  as sumQty, 
                    mean(Quantity) as mean
             FROM dfTable""").show() 

+-------+----------------+
| sumQty|            mean|
+-------+----------------+
|5176450|9.55224954743324|
+-------+----------------+



In [48]:
from pyspark.sql.functions import mean, expr 

In [49]:
df.select(
  count("Quantity").alias("total_transactions"),
  sum("Quantity").alias("total_purchases"),
  avg("Quantity").alias("avg_purchases"),
  expr("mean(Quantity)").alias("mean_purchases")) \
.selectExpr(
  "total_purchases/total_transactions",
  "avg_purchases",
  "mean_purchases").show()

+--------------------------------------+----------------+----------------+
|(total_purchases / total_transactions)|   avg_purchases|  mean_purchases|
+--------------------------------------+----------------+----------------+
|                      9.55224954743324|9.55224954743324|9.55224954743324|
+--------------------------------------+----------------+----------------+



In [51]:
# Example 4 - varience and standard deviation
from pyspark.sql.functions import var_pop, stddev_pop, variance, stddev
from pyspark.sql.functions import var_samp, stddev_samp

In [52]:
df.select(variance("Quantity"), stddev("Quantity"),
  var_pop("Quantity"), var_samp("Quantity"),
  stddev_pop("Quantity"), stddev_samp("Quantity")).show()

+------------------+---------------------+-----------------+------------------+--------------------+---------------------+
|var_samp(Quantity)|stddev_samp(Quantity)|var_pop(Quantity)|var_samp(Quantity)|stddev_pop(Quantity)|stddev_samp(Quantity)|
+------------------+---------------------+-----------------+------------------+--------------------+---------------------+
| 47559.39140929852|   218.08115785023364|47559.30364660883| 47559.39140929852|  218.08095663447745|   218.08115785023364|
+------------------+---------------------+-----------------+------------------+--------------------+---------------------+



In [53]:
spark.sql("""SELECT var_pop(Quantity), 
                    var_samp(Quantity),
                    stddev_pop(Quantity), 
                    stddev_samp(Quantity)
             FROM dfTable""").show()

+-----------------+------------------+--------------------+---------------------+
|var_pop(Quantity)|var_samp(Quantity)|stddev_pop(Quantity)|stddev_samp(Quantity)|
+-----------------+------------------+--------------------+---------------------+
|47559.30364660883| 47559.39140929852|  218.08095663447745|   218.08115785023364|
+-----------------+------------------+--------------------+---------------------+



In [55]:
# Example 5 - skewness & kurtosis
from pyspark.sql.functions import skewness, kurtosis

In [56]:
df.select(skewness("Quantity"), kurtosis("Quantity")).show()  

+--------------------+------------------+
|  skewness(Quantity)|kurtosis(Quantity)|
+--------------------+------------------+
|-0.26407557610430366|119768.05495533711|
+--------------------+------------------+



In [57]:
spark.sql("SELECT skewness(Quantity), kurtosis(Quantity) FROM dfTable").show()

+--------------------+------------------+
|  skewness(Quantity)|kurtosis(Quantity)|
+--------------------+------------------+
|-0.26407557610430366|119768.05495533711|
+--------------------+------------------+



In [58]:
# Example 6 - Covariance and Correlation
from pyspark.sql.functions import corr, covar_pop, covar_samp

In [59]:
df.select(corr("InvoiceNo", "Quantity"), covar_samp("InvoiceNo", "Quantity"),
    covar_pop("InvoiceNo", "Quantity")).show()

+-------------------------+-------------------------------+------------------------------+
|corr(InvoiceNo, Quantity)|covar_samp(InvoiceNo, Quantity)|covar_pop(InvoiceNo, Quantity)|
+-------------------------+-------------------------------+------------------------------+
|     4.912186085633174E-4|             1052.7280543896418|            1052.7260778735376|
+-------------------------+-------------------------------+------------------------------+



In [60]:
spark.sql("""SELECT corr(InvoiceNo, Quantity), 
                        covar_samp(InvoiceNo, Quantity),
                        covar_pop(InvoiceNo, Quantity)
                  FROM dfTable""").show()

+-------------------------+-------------------------------+------------------------------+
|corr(InvoiceNo, Quantity)|covar_samp(InvoiceNo, Quantity)|covar_pop(InvoiceNo, Quantity)|
+-------------------------+-------------------------------+------------------------------+
|     4.912186085633174E-4|             1052.7280543896418|            1052.7260778735376|
+-------------------------+-------------------------------+------------------------------+



In [61]:
#- Example 7 - Aggregation on Collections
from pyspark.sql.functions import collect_set, collect_list